1. ищет в PubMed статьи по интересному для
вас запросу и возвращает абстракты этих
статей (можно N первых статей в списке) в
простом текстовом формате (можно записать
в файл);


In [236]:
from Bio import Entrez
import re
Entrez.email = 'thefulmenius@gmail.com'
handle = Entrez.esearch(db = 'pubmed', term = 'RBD AND antibody')
record = Entrez.read(handle)
mshandle = Entrez.efetch(db="pubmed", id=record["IdList"][0:5], rettype="abstract", retmode="text")
print(mshandle.read()) # first abstracts

1. J Med Virol. 2023 Feb 16. doi: 10.1002/jmv.28584. Online ahead of print.

Receptor-binding domain-based SARS-CoV-2 vaccine adjuvanted with cyclic 
di-adenosine monophosphate enhances humoral and cellular immunity in mice.

Germanó MJ(1)(2), Giai C(3)(4), Cargnelutti DE(1)(4)(2), Colombo MI(3)(4)(2), 
Blanco S(5), Konigheim B(5)(2), Spinsanti L(5), Aguilar J(5), Gallego S(5)(2), 
Valdez HA(6)(2), Mackern-Oberti JP(1)(4)(2), Sanchez MV(1)(2).

Author information:
(1)Instituto de Medicina y Biología Experimental de Cuyo (IMBECU), Centro 
Científico Tecnológico Consejo Nacional de Investigaciones Científicas y 
Técnicas (CONICET)-Mendoza, Universidad Nacional de Cuyo, Av. Ruiz Leal s/n. 
CP:5500, Mendoza, Argentina.
(2)Consejo Nacional de Investigaciones Científicas y Técnicas (CONICET), 
Argentina.
(3)Instituto de Histología y Embriología de Mendoza (IHEM), Universidad Nacional 
de Cuyo-(UNCuyo) CONICET, Mendoza, Argentina.
(4)Facultad de Ciencias Médicas, Universidad Nacional de Cuyo,

2. запрашивает в базе нуклеотидных
последовательностей все последовательности
по названию гена для организма по названию
вида и возвращает список ID или информацию
об их количестве;

In [57]:
handle = Entrez.esearch(db = "nucleotide", term = "DRD4 AND Homo sapiens[orgn]")
record = Entrez.read(handle)
print(f"The number of articles is {record['Count']}")

The number of articles is 167


3. ищет ID организма по названию
в базе;

In [58]:
handle = Entrez.esearch(db = "taxonomy", term = "Drosophila melanogaster")
record = Entrez.read(handle)
print(f"The ID of Drosophila melanogaster in the taxonomy database is: {int(record['IdList'][0])}")

The ID of Drosophila melanogaster in the taxonomy database is: 7227


4. запрашивает в базе белковых или
нуклеотидных последовательностей по
названию гена, после чего возвращает
таблицу с UID (в XML это поле называется Id), accession number (в
XML это поле называется Caption), длиной последовательности
(Slen);

In [221]:
handle = Entrez.esearch(db = "nucleotide", term = "DRD4[gene]", retmax='25')
record = Entrez.read(handle)
for rec in record["IdList"]:
    temphandle = Entrez.read(Entrez.esummary(db = "nucleotide", id=rec, retmode="text"))
    print(temphandle[0]["Id"]+"\t"+temphandle[0]['Caption']+"\t"+\
          f"Length: {re.sub('[^1-9]', '', str(temphandle[0]['Length']))}")
    
#First 25 results out of 2000+

1653961229	NM_000797	Length: 1399
2283785239	JAMQTK000000000	Length: 2544
2270577982	JAMXTM000000000	Length: 5994
296179407	NG_021241	Length: 142
2310839539	NM_007878	Length: 1278
2310839531	NM_001412325	Length: 1164
2310839498	NM_001412326	Length: 145
864510057	NM_001310167	Length: 174
815891323	NM_012944	Length: 1416
1946563154	MT982613	Length: 619
1946563152	MT982612	Length: 619
1946563150	MT982611	Length: 619
1946563148	MT982610	Length: 619
1946563146	MT982609	Length: 619
2318775854	NC_067202	Length: 19481189
2318773909	XM_051217834	Length: 349
2309190314	JALCYT000000000	Length: 583
2309186809	JALCYT010000002	Length: 1679594
2289973121	OK570083	Length: 1248
2283768710	JAMQTK010000597	Length: 445698
2282102358	MU601396	Length: 113224849
2280970803	JAJAGD000000000	Length: 758
2280970802	JAJHUB000000000	Length: 758
2280955980	JAJHUB010000051	Length: 142474
2280955783	JAJAGD010000051	Length: 142474


5. даёт в базу нуклеотидных или белковых
последовательностей текстовый запрос,
а затем пишет последовательности в файл
в формате fasta (покажите начало файла);

In [224]:
handle = Entrez.esearch(db="protein", term="DRD4 AND Homo sapiens[orgn]")
record = Entrez.read(handle)

Entrez.efetch(db="protein", id=record["IdList"], retmode="text", rettype="fasta").read()
with open("DRD4.fasta", "w") as ouf:
    for rec in record["IdList"]:
        lne = Entrez.efetch(db="protein", id=rec, retmode="text", rettype="fasta").read()
        ouf.write(lne+"\n")
with open("DRD4.fasta", "r") as fastaf:
    snippet = [next(fastaf) for x in range(5)]
    print(snippet)

['>NP_000788.2 D(4) dopamine receptor [Homo sapiens]\n', 'MGNRSTADADGLLAGRGPAAGASAGASAGLAGQGAAALVGGVLLIGAVLAGNSLVCVSVATERALQTPTN\n', 'SFIVSLAAADLLLALLVLPLFVYSEVQGGAWLLSPRLCDALMAMDVMLCTASIFNLCAISVDRFVAVAVP\n', 'LRYNRQGGSRRQLLLIGATWLLSAAVAAPVLCGLNDVRGRDPAVCRLEDRDYVVYSSVCSFFLPCPLMLL\n', 'LYWATFRGLQRWEVARRAKLHGRAPRRPSGPGPPSPTPPAPRLPQDPCGPDCAPPAPGLPRGPCGPDCAP\n']


6. скачивает
белок, соответствующий известному UID нуклеотида;

In [229]:
lhandle = Entrez.elink(dbfrom="nucleotide", db="protein", id="1653961229") 
lrecord = Entrez.read(lhandle)
prothandle = lrecord[0]["LinkSetDb"][0]['Link'][0]['Id']
rrecord = Entrez.efetch(db="protein", id=prothandle, rettype="fasta", retmode="text")
with open("drd4_sapiens.fasta", "w") as ouf:
    ouf.write(rrecord.read()+"\n")
    
with open("drd4_sapiens.fasta", "r") as fastaf:
    snippet = [next(fastaf) for x in range(5)]
    print(snippet)

['>NP_000788.2 D(4) dopamine receptor [Homo sapiens]\n', 'MGNRSTADADGLLAGRGPAAGASAGASAGLAGQGAAALVGGVLLIGAVLAGNSLVCVSVATERALQTPTN\n', 'SFIVSLAAADLLLALLVLPLFVYSEVQGGAWLLSPRLCDALMAMDVMLCTASIFNLCAISVDRFVAVAVP\n', 'LRYNRQGGSRRQLLLIGATWLLSAAVAAPVLCGLNDVRGRDPAVCRLEDRDYVVYSSVCSFFLPCPLMLL\n', 'LYWATFRGLQRWEVARRAKLHGRAPRRPSGPGPPSPTPPAPRLPQDPCGPDCAPPAPGLPRGPCGPDCAP\n']


7.
скачивает все последовательности из
работы с PMID … (например, из первого задания)
и пишет их в файл fasta.

In [243]:
lhandle = Entrez.elink(dbfrom="pubmed", db="nucleotide", id="20558169")
lrecord = Entrez.read(lhandle)
ids = []
for el in lrecord[0]["LinkSetDb"][0]["Link"]:
    ids.append(el['Id'])
rrecord = Entrez.efetch(db="nucleotide", id=ids[:4], rettype="fasta", retmode="text")
with open ("py_fasta_pmid.fasta", "w") as ouf:
    ouf.write(rrecord.read()+"\n")
    
# В моей статье не было нуклеотидных последовательностей